In [1]:
import pandas as pd
import numpy as np

import urllib.request
import io
import time


from scipy.io import arff
from sklearn.model_selection import KFold
from collections import Counter

from sklearn.preprocessing import MinMaxScaler
import re

In [2]:
url = 'http://promise.site.uottawa.ca/SERepository/datasets/kc2.arff'
ftpstream = urllib.request.urlopen(url)
data, meta = arff.loadarff(io.StringIO(ftpstream.read().decode('utf-8')))
kc2 = pd.DataFrame(data)
kc2= kc2.sample(frac=1, random_state=20)


Y = kc2['problems']
X = kc2.drop(columns=['problems'])

#Transforming in boolean class
Y = Y.apply(str).str.replace("b|'", '')
Y = Y.apply(lambda x: 1 if x == 'yes' else 0)

#Normalizing values
scaler = MinMaxScaler(feature_range=(0, 1))
x_scaled = scaler.fit_transform(X)
X = pd.DataFrame(x_scaled)

In [16]:
class Perceptron:
    def __init__(self,learning_rate = 0.01, n_iters=100):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.weights = None
        self.bias = None
    
    def activation_func(self, x):
        return np.where(x>=0, 1, 0)
    
    def fit(self, X, y):
        begin = time.time()
        n_samples, n_features = X.shape
        
        self.weights = np.zeros(n_features)
        self.bias = 0
        
        _y = np.array([1 if i > 0 else 0 for i in y])
        
        for i in range(self.n_iters):
            for idx, x_i in enumerate(X):
                
                linear_output = np.dot(x_i, self.weights[0]) + self.bias
                y_predict = self.activation_func(linear_output)
                
                #update rule
                update = self.lr * (_y[idx] - y_predict)
                
                self.weights += update * x_i
                self.bias += update
        return time.time() - begin
                
    def predict(self, X):
        linear_output = np.dot(X, self.weights) + self.bias
        y_predict = self.activation_func(linear_output)
        return y_predict

In [19]:
def accuracy(y_real, y_pred):
    return np.sum(y_real == y_pred) / len(y_real)

kf = KFold(n_splits=5)
p = Perceptron(learning_rate=0.1, n_iters=1000)
accuracies = []

for train, test in kf.split(X):
    p.fit(X.iloc[train], Y.iloc[train])
    predictions = p.predict(X.iloc[test])
    acc = 
    accuracies.append(accuracy(Y.iloc[test], predictions))

print("Perceptron classification accuracy: ", np.mean(accuracies))



Perceptron classification accuracy:  0.6693040293040293
